# chapter 9 텐서플로우 시작하기

In [1]:
import tensorflow as tf
print("Tensorflow Version: {}".format(tf.__version__))

Tensorflow Version: 1.12.0


* tensorflow 를 시작하면 기본적으로 그래프는 생성되지만 세션은 생성되지 않음
* 그래프의 기본적인 operation을 가져옴

In [2]:
def checkGraph_session_operations():
    print("Default Graph  :", tf.get_default_graph())
    print("Default Session:", tf.get_default_session())
    print("Graph operations : ",tf.get_default_graph().get_operations())
    print()

###### Scikit Learn과 호환되는 간결한 라이브러리 TF.Learn (tf.contrib.learn) 제공
* tf.contrib.learn <-- deprecated. 대신 1.8버전부터는 tf.estimator로 깔끔하게 정리되어 이동됨
* 참고: https://excelsior-cjh.tistory.com/157
* tf.contrib.learn : https://www.tensorflow.org/api_docs/python/tf/contrib/learn
* tf.estimator : https://www.tensorflow.org/versions/r1.12/api_docs/python/tf/estimator

###### 신경망의 구축, 훈련, 평가를 단순화한 TF-slim API 제공 
* https://www.tensorflow.org/api_docs/python/tf/contrib/slim
###### Keras나 Preety Tensor와 같은 고수준 API가 텐서플로우 기반으로 독립적으로 구축
* https://www.tensorflow.org/api_docs/python/tf/keras
###### 기본 파이썬 API는 복잡도가 높긴 하지만 매우 유연함 --> 다양한 신경망 구조 구현 가능
###### Loss 값을 최소화하는 모델 파라미터를 찾기 위한 고수준의 최적화 노드제공
###### 자동 미분 (autodiff) 지원 
###### 사용자가 직접 정의한 그래디언트를 자동으로 계산
###### 클라우드 서비스
* https://cloud.google.com/ml-engine/
###### 풍부한 튜토리얼 및 리소스 
* https://github.com/jtoy/awesome-tensorflow 

### [보스턴 housing price 데이터셋 예제]

##### 빌드 단계

In [3]:
from sklearn import datasets, metrics, preprocessing

# 학습 데이터 가져오기 
boston = datasets.load_boston()
print("Boston Type : {}".format(type(boston)))

x_data = preprocessing.StandardScaler().fit_transform(boston.data)
y_data = boston.target
print("x_data Type : {}".format(type(x_data)))
print("y_data Type : {}".format(type(y_data)))
print('x_data.shape :', x_data.shape)
print('y_data.shape :', y_data.shape)

Boston Type : <class 'sklearn.utils.Bunch'>
x_data Type : <class 'numpy.ndarray'>
y_data Type : <class 'numpy.ndarray'>
x_data.shape : (506, 13)
y_data.shape : (506,)


In [4]:
checkGraph_session_operations()

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0xb2d554898>
Default Session: None
Graph operations :  []



* Linear Regression: Natvie TensorFlow

In [5]:
# 변수 선언
x = tf.placeholder(tf.float64, shape=(None, 13))
y_true = tf.placeholder(tf.float64, shape=(None))

w = tf.Variable(initial_value=tf.zeros([1, 13], dtype=tf.float64, name='weights'))
b = tf.Variable(initial_value=0, dtype=tf.float64, name='bias')

In [6]:
checkGraph_session_operations()

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0xb2d554898>
Default Session: None
Graph operations :  [<tf.Operation 'Placeholder' type=Placeholder>, <tf.Operation 'Placeholder_1' type=Placeholder>, <tf.Operation 'weights' type=Const>, <tf.Operation 'Variable' type=VariableV2>, <tf.Operation 'Variable/Assign' type=Assign>, <tf.Operation 'Variable/read' type=Identity>, <tf.Operation 'bias/initial_value' type=Const>, <tf.Operation 'bias' type=VariableV2>, <tf.Operation 'bias/Assign' type=Assign>, <tf.Operation 'bias/read' type=Identity>]



In [7]:
# 가설수식
y_pred = tf.matmul(w, tf.transpose(x)) + b
# 비용함수      
loss = tf.reduce_mean(tf.square(y_true-y_pred))  # MSE
# 최적화 함수
learningRate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learningRate)
train = optimizer.minimize(loss)

In [8]:
checkGraph_session_operations()

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0xb2d554898>
Default Session: None
Graph operations :  [<tf.Operation 'Placeholder' type=Placeholder>, <tf.Operation 'Placeholder_1' type=Placeholder>, <tf.Operation 'weights' type=Const>, <tf.Operation 'Variable' type=VariableV2>, <tf.Operation 'Variable/Assign' type=Assign>, <tf.Operation 'Variable/read' type=Identity>, <tf.Operation 'bias/initial_value' type=Const>, <tf.Operation 'bias' type=VariableV2>, <tf.Operation 'bias/Assign' type=Assign>, <tf.Operation 'bias/read' type=Identity>, <tf.Operation 'transpose/Rank' type=Rank>, <tf.Operation 'transpose/sub/y' type=Const>, <tf.Operation 'transpose/sub' type=Sub>, <tf.Operation 'transpose/Range/start' type=Const>, <tf.Operation 'transpose/Range/delta' type=Const>, <tf.Operation 'transpose/Range' type=Range>, <tf.Operation 'transpose/sub_1' type=Sub>, <tf.Operation 'transpose' type=Transpose>, <tf.Operation 'MatMul' type=MatMul>, <tf.Operation 'add' type=Add>, <tf.Opera

##### 실행단계

In [9]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for step in range(300):
        MSE, _ = sess.run([loss, train], feed_dict={x: x_data, y_true: y_data})
        if (step + 1) % 40 == 0:
            print('Step: {:2d}\t MSE: {:.5f}'.format(step + 1, MSE))
            
    loss = sess.run(loss, feed_dict={x: x_data, y_true: y_data})
    print('\nMSE: {0:.5f}'.format(loss))
    print("Default Graph  :", tf.get_default_graph())
    print("Default Session:", tf.get_default_session())
    print()
    
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
#print("Graph operations : ",tf.get_default_graph().get_operations())
print()

Step: 40	 MSE: 22.29875
Step: 80	 MSE: 21.99908
Step: 120	 MSE: 21.92992
Step: 160	 MSE: 21.90730
Step: 200	 MSE: 21.89931
Step: 240	 MSE: 21.89644
Step: 280	 MSE: 21.89541

MSE: 21.89517
Default Graph  : <tensorflow.python.framework.ops.Graph object at 0xb2d554898>
Default Session: <tensorflow.python.client.session.Session object at 0x1a2e347da0>

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0xb2d554898>
Default Session: None



### [Linear Regression: tf.estimator 사용예제]

In [10]:
"""
feature_column = [tf.feature_column.numeric_column(key='x', shape=13)]

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'x': x_data}, y_data, shuffle=True, batch_size=506, num_epochs=300
)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'x': x_data}, y_data, shuffle=True, batch_size=506, num_epochs=1
)

reg = tf.estimator.LinearRegressor(
    feature_columns=feature_column,
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.001),
    loss_reduction=tf.losses.Reduction.MEAN
)

reg.train(input_fn=train_input_fn)

MSE = reg.evaluate(input_fn=eval_input_fn)

print(MSE)

print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
# """

'\nfeature_column = [tf.feature_column.numeric_column(key=\'x\', shape=13)]\n\ntrain_input_fn = tf.estimator.inputs.numpy_input_fn(\n    {\'x\': x_data}, y_data, shuffle=True, batch_size=506, num_epochs=300\n)\n\neval_input_fn = tf.estimator.inputs.numpy_input_fn(\n    {\'x\': x_data}, y_data, shuffle=True, batch_size=506, num_epochs=1\n)\n\nreg = tf.estimator.LinearRegressor(\n    feature_columns=feature_column,\n    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.001),\n    loss_reduction=tf.losses.Reduction.MEAN\n)\n\nreg.train(input_fn=train_input_fn)\n\nMSE = reg.evaluate(input_fn=eval_input_fn)\n\nprint(MSE)\n\nprint("Default Graph  :", tf.get_default_graph())\nprint("Default Session:", tf.get_default_session())\n# '

* 그래프 초기화(삭제하고 다시 생성한듯...) 하여 기존에 담겨 있던 내용을 초기화 함

In [11]:
tf.reset_default_graph()

In [12]:
checkGraph_session_operations()

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0xb2d5546a0>
Default Session: None
Graph operations :  []



# session 생성시 차이점( with tf.Session  VS sess = tf.Session() )

-  with 블록을 활용한 default session 지정
  - with 블록 내부의 sess는 default session
  - with 블록이 끝나면 sess는 자동 종료
  
- default session이 생성된 상황에서는 run() 호출 가능
  - x.initializer.run() == tf.get_default_session().run(x.initializer)
  - f.eval() == tf.get_default_session().run(f)


###### default session 에서 InteractiveSession을 생성하면 default session 이 InteractiveSession으로 변경됨

---
## 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [14]:
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0xb2d554518>
Default Session: None


In [15]:
tf.reset_default_graph()
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
print(f)

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None
Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None
Tensor("add_1:0", shape=(), dtype=int32)


In [16]:
sess = tf.Session()

sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())

sess.close()

42
Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None


-  with 블록을 활용한 default session 지정
  - with 블록 내부의 sess는 default session
  - with 블록이 끝나면 sess는 자동 종료
  
- default session이 생성된 상황에서는 run() 호출 가능
  - x.initializer.run() == tf.get_default_session().run(x.initializer)
  - f.eval() == tf.get_default_session().run(f)

In [17]:
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
print()

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print("Default Graph  :", tf.get_default_graph())
    print("Default Session:", tf.get_default_session())
    print(sess is tf.get_default_session())  
    print()

print(result)
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: <tensorflow.python.client.session.Session object at 0x1a2e57a780>
True

42
Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None


In [18]:
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
print()

with tf.Session() as sess:
    tf.get_default_session().run(x.initializer)
    tf.get_default_session().run(y.initializer)
    result = tf.get_default_session().run(f)
    print("Default Graph  :", tf.get_default_graph())
    print("Default Session:", tf.get_default_session())
    print(sess is tf.get_default_session())  
    print()

print(result)
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: <tensorflow.python.client.session.Session object at 0x1a2e57a860>
True

42
Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None


* 각 변수를 개별적으로 초기화하는 대신 global_variables_initialzer() 함수 사용

In [19]:
init = tf.global_variables_initializer()

with tf.Session():
    init.run()   # 실제 모든 변수 초기화
    result = f.eval()

print(result)
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())

42
Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None


- InteractiveSession 사용
  - InteractiveSession이 생성되면 생성된 session 이 default session 으로 지정됨

In [20]:
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
print()
init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
print()

init.run()
result = f.eval()
sess.close()

print(result)
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: <tensorflow.python.client.session.InteractiveSession object at 0x1a2e57a518>

42
Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None


---
## 9.3 계산 그래프 관리

- 노드를 만들면 자동으로 default graph에 추가

In [22]:
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
print()

tf.reset_default_graph()
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
print()

x1 = tf.Variable(1)
print(x1.graph is tf.get_default_graph())

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56f1d0>
Default Session: None

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56fb00>
Default Session: None

True


In [23]:
print("Default Graph  :", tf.get_default_graph())
print("Default Session:", tf.get_default_session())
print()

graph = tf.Graph()
print(graph is tf.get_default_graph())

with graph.as_default():
    x2 = tf.Variable(2)
    print(graph is tf.get_default_graph())    
    print(x2.graph is tf.get_default_graph())
    
print(x2.graph is tf.get_default_graph()) # graph (== x2.graph)는 디폴트 그래프에서 분리된 다른 그래프
print(x2.graph is graph)

Default Graph  : <tensorflow.python.framework.ops.Graph object at 0x1a2e56fb00>
Default Session: None

False
True
True
False
True


- ***[주의]***
  - Tensorflow 코딩을 하는 동안 default graph에 중복되거나 불필요한 노드가 포함될 수 있음
  - 그러므로, 코드 시작시에 tf.reset_default_graph()로 default graph를 초기화하는 습관 필요

---
## 9.4 노드 값의 생애주기

- 임의의 노드 평가 --> 이 노드가 의존하고 있는 다른 노드들을 자동으로 먼저 찾아 평가

In [24]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

print(w)
print(x)
print(y)
print(z)
print()
with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

Tensor("Const:0", shape=(), dtype=int32)
Tensor("add:0", shape=(), dtype=int32)
Tensor("add_1:0", shape=(), dtype=int32)
Tensor("mul:0", shape=(), dtype=int32)

10
15


- 위 코드의 단점
  - w와 x가 두번 평가됨
  - 즉, 상수 노드에 대해서는 평가된 노드의 값은 한번의 eval() 실행이후 세션 내에서 사라짐
- 아래 코드는 개선된 코드
  - sess.run()을 통해서 한번 그래프 실행으로 w와 x는 한번만 평가됨

In [25]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


- 변수 노드에 대해서는 eval() 이후에도 세션 내에 유지됨
- 변수의 일생
  - 변수가 초기화될 때 시작
  - 세션이 종료가 되면 내용이 사라짐

- 단일 프로세스 텐서플로우에서는 여러 세션에서 동일한 그래프를 재사용하더라도 어떤 상태도 공유하지 않음

In [28]:
with tf.Session() as sess:
    y_val = sess.run([y])
    print(y_val)  # 10
    z_val = sess.run((z))
    print(z_val)
    x_val = sess.run(x)
    print(z_val)
    

[10]
15
15


---
## 9.5 텐서플로우를 이용한 선형 회귀

In [31]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf

housing = fetch_california_housing()
print(type(housing))
print(housing.data.shape)

m = housing.data.shape[0]

# 각 훈련 샘플에 편향 특성(x_0 = 1) 추가
# np.c_ : (concat) 배열 합치기 : https://rfriend.tistory.com/352
housing_data_plus_bias = np.c_[np.ones(shape=(m, 1)), housing.data]
print(housing_data_plus_bias.shape)

print()

print(housing.target.shape)
housing_target = housing.target.reshape(-1, 1)
print(housing_target.shape)

<class 'sklearn.utils.Bunch'>
(20640, 8)
(20640, 9)

(20640,)
(20640, 1)


In [32]:
tf.reset_default_graph()

X = tf.constant(housing_data_plus_bias, dtype=tf.float64, name="X")
y = tf.constant(housing_target, dtype=tf.float64, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    result = theta.eval()

print(result)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


---
## 9.6 경사 하강법 구현
- 정규 방정식 대신 경사 하강법을 활용하여 $\theta$를 구하기
- 경사 하강법 구현 방법
  - 1) 직접 gradient 계산
  - 2) Tensorflow의 자동 미분 (autodiff) 사용
  - 3) Tensorflow의 내장 optimizer 사용

- 경사 하강법 사용시에는 입력 특성 정규화 중요
  - scikit-learn의 StandardScaler 사용

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [37]:
# 학습 데이터의 컬럼별 평균
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
(20640, 9)


### 9.6.1 직접 gradient 계산

- Mean Sqaure Error (MSE)
$$J(\theta)= \frac{1}{m} \sum(X \cdot \theta - y)^2$$
- Gradient of MSE 
$$J'(\theta) = \frac{2}{m} X^T \cdot (X \cdot \theta - y)$$
  - $X^T$: (9, 20640)
  - $X$: (20640, 9) <-- scaled_housing_data_plus_bias
  - $\theta$: (9, 1) 
  - $X \cdot \theta$: (20640, 1)
  - $y$: (20640, 1) <-- housing_target
  
  
- Gradient descent for getting $\theta$
  - $$\theta = \theta - \alpha J'(\theta)$$

In [38]:
tf.reset_default_graph()

learning_rate = 0.01

m, n = scaled_housing_data_plus_bias.shape
print("Number of samples: {0}, Number of features: {1}".format(m, n))
print()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_target, dtype=tf.float32, name="y")

theta = tf.Variable(
    tf.random_uniform(shape=[n, 1], minval=-1.0, maxval=1.0, seed=42), 
    name="theta"
)

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

n_epochs = 1000

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

print("Best theta: \n{0}".format(best_theta))

Number of samples: 20640, Number of features: 9

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.53737885
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293705
Best theta: 
[[ 2.06855226e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44081801e-04]
 [-3.91945131e-02]
 [-8.61356556e-01]
 [-8.23479712e-01]]


### 9.6.2 직접 gradient 계산
- 후진 모드 자동 미분 (Backword Propagation - Autodiff)
- ***gradients = tf.gradients(mse, [theta])[0]***

In [39]:
x = tf.constant(2.0)
y = x**2 + x - 1

grad = tf.gradients(y, x)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    grad_value = sess.run(grad)
    print(grad_value)

[5.0]


In [40]:
tf.reset_default_graph()

learning_rate = 0.01

m, n = scaled_housing_data_plus_bias.shape
print("Number of samples: {0}, Number of features: {1}".format(m, n))
print()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_target, dtype=tf.float32, name="y")

theta = tf.Variable(
    tf.random_uniform(shape=[n, 1], minval=-1.0, maxval=1.0, seed=42), 
    name="theta"
)

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

###
gradients = tf.gradients(mse, theta)[0]
###
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

n_epochs = 1000

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

print("Best theta: \n{0}".format(best_theta))

Number of samples: 20640, Number of features: 9

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704
Best theta: 
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


### 9.6.3 Optimizer 사용


- GradientDescentOptimizer 사용

In [41]:
tf.reset_default_graph()

learning_rate = 0.01

m, n = scaled_housing_data_plus_bias.shape
print("Number of samples: {0}, Number of features: {1}".format(m, n))
print()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_target, dtype=tf.float32, name="y")

theta = tf.Variable(
    tf.random_uniform(shape=[n, 1], minval=-1.0, maxval=1.0, seed=42), 
    name="theta"
)

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

###
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
###

init = tf.global_variables_initializer()

n_epochs = 1000

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

print("Best theta: \n{0}".format(best_theta))

Number of samples: 20640, Number of features: 9

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704
Best theta: 
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


- MomentumOptimizer 사용

In [42]:
tf.reset_default_graph()

learning_rate = 0.01

m, n = scaled_housing_data_plus_bias.shape
print("Number of samples: {0}, Number of features: {1}".format(m, n))
print()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_target, dtype=tf.float32, name="y")

theta = tf.Variable(
    tf.random_uniform(shape=[n, 1], minval=-1.0, maxval=1.0, seed=42), 
    name="theta"
)

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

###
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
###

init = tf.global_variables_initializer()

n_epochs = 1000

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

print("Best theta: \n{0}".format(best_theta))

Number of samples: 20640, Number of features: 9

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.52731586
Epoch 200 MSE = 0.52441466
Epoch 300 MSE = 0.52432835
Epoch 400 MSE = 0.5243219
Epoch 500 MSE = 0.524321
Epoch 600 MSE = 0.52432066
Epoch 700 MSE = 0.524321
Epoch 800 MSE = 0.52432126
Epoch 900 MSE = 0.524321
Best theta: 
[[ 2.068558  ]
 [ 0.8296167 ]
 [ 0.11875112]
 [-0.26552212]
 [ 0.30569226]
 [-0.00450316]
 [-0.03932616]
 [-0.8998917 ]
 [-0.87054664]]
